In [1]:
from statistics import stdev, mean
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import numpy as np
import random
import scipy
import math
import copy
import csv

In [ ]:
def import_csv():  
    
    """Считывание данных из файла
    
    Метод считывает данные из CSV файла и преобразует их в DataFrame.
    
    """
    
    try:
        with open(input('Введите путь к файлу: \n'), encoding='utf-8-sig') as data_file:
            csv = []
            A = []
            for line in data_file:
                csv = line.strip().split(';')
                csv = list(map(lambda x: x.replace(',', '.') if ',' in x else x, csv))
                try:
                    csv = list(map(lambda x: float(x), csv))
                except:
                    pass
                csv = [(x) for x in csv]
                A.append(csv)
            A = A[1:]
            A = pd.DataFrame(A, 
                             columns=['x', 'y'], 
                             index=[i for i in range(1, len(A) + 1)])
            print('\nВы импортировали: \n', A)
            
            A = A.sort_values(by='x')
            A.index = pd.Series([i for i in range(1, len(A) + 1)])
        return A
    except FileNotFoundError:
        print('Искомый файл не найден! Попробуйте еще раз: \n')
        return import_csv()


def mean_value(value):
    """Расчет среднего значения массива."""
    return mean(value)


def covariance(x, y):
    """Расчет ковариации по выборке."""
    dx = x - mean_value(x)
    dy = y - mean_value(y)
    return (dx * dy).sum() / dx.count()


def var_value(value):
    """Расчет дисперсии по выборке."""
    n = len(value)
    mean = mean_value(value)
    return sum((item - mean)**2 for item in value) / n


def approxim_line(num):
    
    """Расчет аппроксимирующей линии и коэффициентов k и b
    
    Метод формирует массив аппроксимирующей линии и добавляет его в виде столбца в таблицу DataFrame.
    Также высчитываются коэффициенты k и b линейной аппроксимирующей функции.
    
    """
    
    column_x_mean = mean_value(num['x'])
    column_y_mean = mean_value(num['y'])
    
    k = covariance(num['x'], num['y']) / var_value(num['x'])
    b = column_y_mean - column_x_mean*k
    
    approxim_line_ = [k*x + b for x in num['x']]
    num['Аппроксимирующая линия'] = approxim_line_
    return num, k, b, column_x_mean, column_y_mean


def equation_linal_approxim(k, b):
    """Формирование уравнения линейной аппроксимации."""
    return f'y = {round(k, 4)}x + {round(b, 4)}'


def graph_linal_approxim(num, l, k, b):
    """Метод строит график линейной аппроксимации и строит прямые доверительных интервалов."""
    def fun_sigma(b, color, linestyle, label, l):
        #x = np.linspace(num['x'][1], num['x'][l], 5)
        y = [k*i + b for i in l]
        plt.plot(l, y,
                 color=color,
                 linestyle=linestyle, 
                 label=label)
    
    plt.figure(figsize=(14, 8))

    plt.plot(num['x'], num['y'], 'o',
             color='#1e5356', 
             label='y')
    
    fun_sigma(b, 'black', 'solid', equation_linal_approxim(k, b), l)
    fun_sigma(b + stdev(num['y']), '#054907', '--', equation_linal_approxim(k, b + stdev(num['y'])), l)
    fun_sigma(b - stdev(num['y']), '#800000', '--', equation_linal_approxim(k, b - stdev(num['y'])), l)
    fun_sigma(b + 2*stdev(num['y']), '#008000', '-.', equation_linal_approxim(k, b + 2*stdev(num['y'])), l)
    fun_sigma(b - 2*stdev(num['y']), '#A52A2A', '-.', equation_linal_approxim(k, b - 2*stdev(num['y'])), l)
    fun_sigma(b + 3*stdev(num['y']), '#15B01A', ':', equation_linal_approxim(k, b + 3*stdev(num['y'])), l)
    fun_sigma(b - 3*stdev(num['y']), '#FF0000', ':', equation_linal_approxim(k, b - 3*stdev(num['y'])), l)
    
    plt.title(f'Линейная аппроксимация {equation_linal_approxim(k, b)}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid()

    plt.legend(fontsize=12,
              bbox_to_anchor=(1, 1))
    plt.show()
    

def dispersion(num, column):
    """Расчет дисперсии по выборке."""
    sigma = sum([(k - n)**2 for k, n in zip(num['y'], num[column])])
    sigma = round(math.sqrt(sigma) / len(num['x']), 4)
    return sigma


def free_values(num):
    """Расчет свободных членов."""
    x1 = sum(x**2 * y for x, y in zip(num['x'], num['y']))
    x2 = sum(x * y for x, y in zip(num['x'], num['y']))
    x3 = sum(num['y'])
    return [x1, x2, x3]


def x_degree(num):
    """Формирование массива из суммы степеней x."""
    return [sum([i**a for i in num['x']]) for a in range(5)]


def matrix(num):
    """Расчет коэффициентов a, b, c квадратичной аппроксимирующей функции."""
    x = x_degree(num)
    matrix_reverse = np.linalg.inv([
        [x[4], x[3], x[2]],
        [x[3], x[2], x[1]],
        [x[2], x[1], x[0]]
    ])
    matrix = np.dot(matrix_reverse, free_values(num))
    return matrix[0], matrix[1], matrix[2]


def equation_quadratic_approxim(num, a, b, c):
    """Формирование уравнения квадратичной аппроксимации."""
    return f'y = {round(a, 4)}x\u00B2 + {round(b, 4)}x + {round(c, 4)}'
    
    
def approxim_curve(num):
    """Расчет аппроксимирующей кривой."""
    a, b, c = matrix(num)
    approxim_curve_ = [a*(x**2) + b*x + c for x in num['x']]
    num['Аппроксимирующая кривая'] = approxim_curve_
    return num


def delete_release(num):
    """Метод удаляет выбросы и возвращает выборку без них."""
    num_without_release = copy.deepcopy(num)
    for n, i in enumerate(num['y'], 1):
        upper_line = num['Аппроксимирующая линия'][n] + 1.5*stdev(num['y'])
        bottom_line = num['Аппроксимирующая линия'][n] - 1.5*stdev(num['y'])
        
        if i >= upper_line or i <= bottom_line:
            num_without_release.drop(labels=[n], axis=0, inplace=True)
    
    approxim_line(num_without_release)
    approxim_curve(num_without_release)
    
    return num_without_release


def graph_quadratic_approxim(num, l):
    """Метод строит график квадратичной аппроксимации и строит прямые доверительных интервалов."""
    def fun_sigma(c, color, linestyle, label, l):
        #x = np.linspace(num['x'][1], num['x'][l], 50)
        y = [a*(i**2) + b*i + c for i in l]
        plt.plot(l, y,
                 color=color,
                 linestyle=linestyle, 
                 label=label)
    
    a, b, c = matrix(num)
    
    plt.figure(figsize=(14, 8))

    plt.plot(num['x'], num['y'], 'o',
             color='#1e5356', 
             label='y')
    
    fun_sigma(c, 'black', 'solid', equation_quadratic_approxim(num, a, b, c), l)
    fun_sigma(c + stdev(num['y']), '#054907', '--', equation_quadratic_approxim(num, a, b, c + stdev(num['y'])), l)
    fun_sigma(c - stdev(num['y']), '#800000', '--', equation_quadratic_approxim(num, a, b, c - stdev(num['y'])), l)
    fun_sigma(c + 2*stdev(num['y']), '#008000', '-.', equation_quadratic_approxim(num, a, b, c + 2*stdev(num['y'])), l)
    fun_sigma(c - 2*stdev(num['y']), '#A52A2A', '-.', equation_quadratic_approxim(num, a, b, c - 2*stdev(num['y'])), l)
    fun_sigma(c + 3*stdev(num['y']), '#15B01A', ':', equation_quadratic_approxim(num, a, b, c + 3*stdev(num['y'])), l)
    fun_sigma(c - 3*stdev(num['y']), '#FF0000', ':', equation_quadratic_approxim(num, a, b, c - 3*stdev(num['y'])), l)
    
    plt.title(f'Квадратичнаяя аппроксимация {equation_quadratic_approxim(num, a, b, c)}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid()

    plt.legend(fontsize=12,
               bbox_to_anchor=(1, 1))
    plt.show()


def graph_all_func(num, k, b, num_without_release, k_cl, b_cl):
    
    """
    Метод отображает графики линейной и нелинейной аппроксимации 
    в исходном виде и после удаления выбросов.
    
    """
    
    def linal(num, l, k, b, color):
        y = [k*i + b for i in l]
        plt.plot(l, y,
                 color=color,
                 linestyle='solid', 
                 label=equation_linal_approxim(k, b))
    
    def quadratic(num, l, ak, bk, ck, color):
        y = [ak*(i**2) + bk*i + ck for i in l]
        plt.plot(l, y,
                 color=color,
                 linestyle='solid', 
                 label=equation_quadratic_approxim(num, ak, bk, ck))
    
    plt.figure(figsize=(14, 8))
    plt.plot(num['x'], num['y'], 'o',
             color='#1e5356',
             label='y')
    
    ak, bk, ck = matrix(num)
    ak_cl, bk_cl, ck_cl = matrix(num_without_release)
    
    l = np.linspace(num['x'][1], num['x'][len(num['x'])], 50)
    
    linal(num, l, k, b, 'g')
    linal(num_without_release, l, k_cl, b_cl, 'b')
    quadratic(num, l, ak, bk, ck, 'r')
    quadratic(num_without_release, l, ak_cl, bk_cl, ck_cl, 'y')
    
    plt.title(f'Аппроксимация')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid()

    plt.legend(fontsize=12,
               bbox_to_anchor=(1, 1))
    plt.show()


def graphs_approxim(num):
    """Расчет квадратичной аппроксимизации и построение графика функции."""
    approxim_line_table, k, b, column_x_mean, column_y_mean = approxim_line(num)
    ak, bk, ck = matrix(num)
    l = np.linspace(num['x'][1], num['x'][len(num['x'])], 50)
    
    print('\n\nДанные\n', approxim_curve(num))
    print(f'\nАппроксимирующая линейная функция: {equation_linal_approxim(k, b)}')
    print('Дисперсия: {}'.format(dispersion(num, 'Аппроксимирующая линия')))
    print(f'\nАппроксимирующая квадратная функция: {equation_quadratic_approxim(num, ak, bk, ck)}')
    print('Дисперсия: {}'.format(dispersion(num, 'Аппроксимирующая кривая')))
    
    num_without_release = delete_release(num)
    approxim_line_table_cl, k_cl, b_cl, column_x_mean, column_y_mean = approxim_line(num_without_release)
    ak_cl, bk_cl, ck_cl = matrix(num_without_release)
    print('\n\nВыборка без выбросов:\n', num_without_release)
    print(f'\nАппроксимирующая линейная функция: {equation_linal_approxim(k_cl, b_cl)}')
    print('Дисперсия: {}'.format(dispersion(num_without_release, 'Аппроксимирующая линия')))
    print(f'\nАппроксимирующая квадратная функция: {equation_quadratic_approxim(num_without_release, ak_cl, bk_cl, ck_cl)}')
    print('Дисперсия: {}'.format(dispersion(num_without_release, 'Аппроксимирующая кривая')))
    graph_all_func(num, k, b, num_without_release, k_cl, b_cl)
    graph_linal_approxim(num, l, k, b)
    graph_linal_approxim(num_without_release, l, k_cl, b_cl)
    graph_quadratic_approxim(num, l)
    graph_quadratic_approxim(num_without_release, l)


num = import_csv()
graphs_approxim(num)